In [40]:
# class for invalid API Key exception
class APIKeyException(Exception): 
    def __init__(self, message):
        self.message = message  # custom error message for many types of errors (api key not found, api key invalid, rate limited, etc)

In [41]:
import pickle           # to load models
import requests         # to make an API request and get recent data
import pandas as pd     # for data
import numpy as np      # for math

DATACOUNT = 20          # SHOULD BE THE SAME AS IN getdata.py
CLASSIFYSPLIT = 0.20    # SHOULD BE THE SAME AS IN train.py
PATH = "models/"        # directory of the models

# gets API key if present
APIKEY = ""     # ENTER YOUR API KEY HERE
if not APIKEY:
    raise APIKeyException("Please enter an API key!")

# loads model and scaler
with open(f"{PATH}model.pkl", "rb") as f:
    model = pickle.load(f)
with open(f"{PATH}scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [ ]:
# returns percent change 
def calcPercentChange(open, close):
    return (float(close)) / float(open)

# gets stock
symbol = input("Enter your stock symbol which you are analyzing: ")

# gets the information for the stock
url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "apikey": APIKEY,
    "symbol": symbol,
    "datatype": "json"
}
data = requests.get(url, params=params).json()

# try to look at data otherwise api key is limited
try:
    data = data["Time Series (Daily)"]
except KeyError:
    try: 
        data["Error Message"]       # checks if there is an error message
        raise APIKeyException("Your API key is invalid.")
    except KeyError:                         # otherwise it must be a rate limit
        raise APIKeyException("You have been rate limited. Try again later.")

# gets most recent data and reverses it
data = list(data.values())[:int(DATACOUNT * (1-CLASSIFYSPLIT)):][::-1]  # TURNS THE DATA INTO THE MOST RECENT TRADING DAYS

# gets array of percent changes in the stock
gains = [calcPercentChange(open=day["1. open"], close=day["4. close"]) for day in data]
volumes = [int(day["6. volume"]) for day in data]

frame = pd.DataFrame({      # PARAMETERS HERE SHOULD MATCH THOSE IN train.py
    "gainVariation": [np.std(gains)],
    "volume": [np.average(volumes)],
    "avgDailyIncrease": [np.average(gains)],
    "overallIncrease": [calcPercentChange(open=data[0]["1. open"], close=data[-1]["4. close"])]
})

# scales dataframe
frame = scaler.transform(frame)

# predicts outcome
outcome = list(model.predict(frame))[0]
probability = list(model.predict_proba(frame))[0][0]

# prints output
print(f"You should {['sell', 'buy'][outcome]} {symbol} with a certainty of {round(probability, 3)}.")